In [1]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd
import numpy as np

import tensorflow as tf

import datetime

from Rakuten_preprocessing import Rakuten_img_path

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier

from src.utils.load import load_classifier
from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

from sklearn.linear_model import LogisticRegression

2024-03-14 19:58:04.188106: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 19:58:04.291309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 19:58:04.291381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 19:58:04.295487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 19:58:04.326907: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)
    
#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [3]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [4]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X_train_90 = pd.concat([X_train, X_test[:round(len(X_test)/2)]], axis=0).reset_index(drop=True)
X_test_90 = X_test[round(len(X_test)/2):].reset_index(drop=True)
y_train_90 = pd.concat([y_train, y_test[:round(len(X_test)/2)]], axis=0).reset_index(drop=True)
y_test_90 = y_test[round(len(X_test)/2):].reset_index(drop=True)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## Fusion model with transformers

In [5]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_fusion_TF_90.csv'

#type of modality
modality = 'fusion'

#Type of classifier
class_type = 'TFmultiClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = 27
max_length = 256
n_epochs = 8
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_min=1e-6
lr_decay_rate = 0.8

callbacks = []
#adding earlystopping callback
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))
#Adding tensorboard callback as the last one
callbacks.append(('TensorBoard', {'log_dir': np.nan, 'histogram_freq': 1, 'update_freq': 'epoch'}))

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#name of previously saved models to use as base estimators
base_name_list = ['camembert-base vit_b16']

#number of transformer blocks after fusion
transfo_numblocks = 6 #3 #0 #8
attn_numheads = 12 #8

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
  #Adjusting tensorboard log directory
  log_dir = os.path.join(config.path_to_tflogs, base_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  callbacks[-1][1]['log_dir'] = log_dir
  #adding the set of parameters to the list
  [txt_base_name, img_base_name] = base_name.split()
  # params_list.append({'modality': modality,
  #                     'class': class_type,
  #                     'base_name': base_name,
  #                     'model_suffix': 'TF' + str(transfo_numblocks) + '_att' + str(attn_numheads),
  #                     'param_grid': {'transfo_numblocks': transfo_numblocks, 'attention_numheads': attn_numheads, 'drop_rate': drop_rate, 
  #                                     'learning_rate': lr0, 'lr_decay_rate': lr_decay_rate, 'lr_min': lr_min,
  #                                     'max_length': max_length, 'num_class': num_class, 
  #                                     'epochs': n_epochs, 'batch_size': batch_size,
  #                                     'validation_data': (X_test, y_test), 'callbacks': [callbacks],
  #                                     'parallel_gpu': True},
  #                     'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
  #                     })
  params_list.append({'modality': modality,
                      'class': class_type,
                      'base_name': base_name,
                      'model_suffix': 'TF' + str(transfo_numblocks) + '_att' + str(attn_numheads) + '_90',
                      'param_grid': {'transfo_numblocks': transfo_numblocks, 'attention_numheads': attn_numheads, 'drop_rate': drop_rate, 
                                      'learning_rate': lr0, 'lr_decay_rate': lr_decay_rate, 'lr_min': lr_min,
                                      'max_length': max_length, 'num_class': num_class, 
                                      'epochs': n_epochs, 'batch_size': batch_size,
                                      'validation_data': (X_test_90, y_test_90), 'callbacks': [callbacks],
                                      'parallel_gpu': True},
                      'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                      })
  
  
#Running the batch over params_list
# results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)
results = fit_save_all(params_list, X_train=X_train_90, y_train=y_train_90, X_test=X_test_90, y_test=y_test_90, result_file_name = result_file_name)

Fitting:  camembert-base vit_b16 None
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-14 19:58:59.307907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 19:58:59.308407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 19:58:59.487745: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 19:58:59.487836: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 19:58:59.487887: I external/local_xla/xla/stream_executor

Found 32 validated image filenames.


2024-03-14 20:00:12.187063: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/8
INFO:tensorflow:Collective all_reduce tensors: 510 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce IndexedSlices: 3 all_reduces, num_devices =2, group_size = 2, implementation = CommunicationImplementation.NCCL
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Collective all_reduce tensors: 510 all_reduces, num_devices = 2, group_size = 2, implementation = Comm

2024-03-14 20:02:39.631860: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-14 20:02:39.652858: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-14 20:02:43.078516: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-14 20:02:46.563971: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe648251ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-14 20:02:46.564119: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-14 20:02:46.564146: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-14 20:02:46.588915: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set

2389/2389 [==============================] - ETA: 0s - loss: 0.8656 - accuracy: 0.7534INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated image filenames.
Found 32 validated ima

In [6]:
clf = load_classifier('fusion/camembert-base-vit_b16_TF3_att12')

All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



loading weights for multimodal model from  fusion/camembert-base-vit_b16_TF3_att12


In [19]:
from src.utils.load import load_batch_results

df_results = load_batch_results('results_benchmark_fusion_TF_90')
display(df_results)

,modality,class,vectorization,meta_method,classifier,tested_params,best_params,score_test,score_test_cat,conf_mat_test,score_train,fit_time,score_cv_test,score_cv_train,fit_cv_time,probs_test,pred_test,y_test,model_path
0,fusion,TFmultiClassifier,NaN,NaN,camembert-base vit_b16,"{'transfo_numblocks': [6], 'attention_numheads...",NaN,0.901472,"[0.7858243451463791, 0.834307992202729, 0.9847...","[[255, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, ...",0.988003,31628.689639,NaN,NaN,NaN,"[[4.129394073970616e-05, 0.0002569991047494113...","[24, 15, 18, 0, 10, 20, 12, 10, 24, 10, 10, 20...","[24, 15, 18, 0, 10, 20, 12, 10, 24, 9, 10, 20,...",fusion/camembert-base-vit_b16_TF6_att12_90


## Example usage

In [ ]:
#defining callbacks
callbacks = []
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))

model_multi = TFmultiClassifier(txt_base_name='camembert-base', img_base_name='vit_b16', 
                                 max_length=256, img_size=(224, 224, 3), augmentation_params=None,
                                 num_class=27, drop_rate=0.2, attention_numheads=8, transfo_numblocks=3,
                                 epochs=8, batch_size=32, learning_rate=5e-5, validation_data=(X_test, y_test), callbacks=callbacks)

clf.fit(X_train, y_train)
clf.classification_score(X_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-10 13:07:20.850430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.850632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948545: I external/local_xla/xla/stream_executor

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu